In [33]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

### https://www.kaggle.com/datasets/heesoo37/120-years-of-olympic-history-athletes-and-results

In [10]:
data = pd.read_csv('/kaggle/input/120-years-of-olympic-history-athletes-and-results/athlete_events.csv')

In [22]:
data

,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,21,185,82,China,CHN,1992 Summer,1992,1,Barcelona,Basketball,Basketball Men's Basketball,0
1,1,21,185,82,China,CHN,2012 Summer,2012,1,London,Judo,Judo Men's Extra-Lightweight,0
2,1,25,185,82,Denmark,DEN,1920 Summer,1920,1,Antwerpen,Football,Football Men's Football,0
3,1,25,185,82,Denmark/Sweden,DEN,1900 Summer,1900,1,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,1
4,0,27,185,82,Netherlands,NED,1988 Winter,1988,0,Calgary,Speed Skating,Speed Skating Women's 500 metres,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,1,29,175,64,Poland-1,POL,1976 Winter,1976,0,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,0
271112,1,21,175,70,Poland,POL,2014 Winter,2014,0,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",0
271113,1,27,197,93,Poland,POL,2014 Winter,2014,0,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",0
271114,1,31,197,93,Poland,POL,1998 Winter,1998,0,Nagano,Bobsleigh,Bobsleigh Men's Four,0


In [17]:
data.isna().sum()

ID        0
Name      0
Sex       0
Age       0
Height    0
Weight    0
Team      0
NOC       0
Games     0
Year      0
Season    0
City      0
Sport     0
Event     0
Medal     0
dtype: int64

In [18]:
data = data.drop(['ID','Name'],axis=1)

In [13]:
data['Medal'] = data['Medal'].apply(lambda x: 1 if str(x) != 'nan' else 0)

In [15]:
noncols = ['Age','Height','Weight']
for col in noncols :
    data[col] = data.groupby(['Sex','Medal'])[col].apply(lambda x: x.fillna(x.mean()).astype(np.int)).reset_index(drop=True)
    

/tmp/ipykernel_32/4101403022.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data[col] = data.groupby(['Sex','Medal'])[col].apply(lambda x: x.fillna(x.mean()).astype(np.int)).reset_index(drop=True)
/tmp/ipykernel_32/4101403022.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, c

In [21]:
binary_fets = ['Season','Sex']
binary_positive = ['Summer','M']
for col , pos in zip(binary_fets , binary_positive):
    data[col] = data[col].apply(lambda x: 1 if x ==pos else 0)

In [25]:
nominal_cols = ['Team','NOC','Games','Year','City','Sport','Event']
nominal_prefs = ['T','N','G','Y','C','S','E']
for col , prefix in zip(nominal_cols,nominal_prefs):
    dummie = pd.get_dummies(data[col],prefix=prefix).astype(np.int)
    data = pd.concat([data,dummie],axis=1)
    data = data.drop(col,axis=1)

/tmp/ipykernel_32/835274328.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dummie = pd.get_dummies(data[col],prefix=prefix).astype(np.int)
/tmp/ipykernel_32/835274328.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Depr

In [28]:
y = data['Medal']
x = data.drop('Medal',axis=1)

In [30]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [31]:
x_train , x_test , y_train , y_test = train_test_split(x,y,train_size=0.7,random_state=13)

In [32]:
inputs = tf.keras.Input(shape=(x.shape[1]))
x = tf.keras.layers.Dense(64,activation='relu')(inputs)
x = tf.keras.layers.Dense(64,activation='relu')(x)
outputs = tf.keras.layers.Dense(1,activation='sigmoid')(x)

model = tf.keras.Model(inputs,outputs)

model.compile(
    optimizer='adam'
    ,loss='binary_crossentropy'
    ,metrics=tf.keras.metrics.AUC(name='auc')
)

model.fit(
    x_train , 
    y_train,
    validation_split=0.2,
    batch_size=25,
    epochs=100,
    callbacks=tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
)

Epoch 1/100
6073/6073 [==============================] - 27s 4ms/step - loss: 0.3320 - auc: 0.8005 - val_loss: 0.3034 - val_auc: 0.8420
Epoch 2/100
6073/6073 [==============================] - 24s 4ms/step - loss: 0.2706 - auc: 0.8704 - val_loss: 0.2726 - val_auc: 0.8698
Epoch 3/100
6073/6073 [==============================] - 24s 4ms/step - loss: 0.2353 - auc: 0.9028 - val_loss: 0.2544 - val_auc: 0.8872
Epoch 4/100
6073/6073 [==============================] - 24s 4ms/step - loss: 0.2102 - auc: 0.9231 - val_loss: 0.2507 - val_auc: 0.8908
Epoch 5/100
6073/6073 [==============================] - 25s 4ms/step - loss: 0.1927 - auc: 0.9356 - val_loss: 0.2465 - val_auc: 0.8932
Epoch 6/100
6073/6073 [==============================] - 25s 4ms/step - loss: 0.1800 - auc: 0.9441 - val_loss: 0.2509 - val_auc: 0.8987
Epoch 7/100
6073/6073 [==============================] - 24s 4ms/step - loss: 0.1686 - auc: 0.9517 - val_loss: 0.2456 - val_auc: 0.8964
Epoch 8/100
6073/6073 [=========================

In [34]:
y_true = np.array(y_test)
y_pred = np.squeeze(np.array(model.predict(x_test) > 0.5 , dtype=np.int))

2542/2542 [==============================] - 4s 2ms/step


/tmp/ipykernel_32/2325219963.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_pred = np.squeeze(np.array(model.predict(x_test) > 0.5 , dtype=np.int))


In [36]:
print('Confusion Matrix \n\n\n ',confusion_matrix(y_true,y_pred))

Confusion Matrix 


  [[68122  1265]
 [ 5119  6829]]
